In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("../heart.csv")

# Features and target variable
X = data.drop(columns=['HeartDisease'])  # Remove the target column from feature

In [2]:
import flask

In [3]:
!pip uninstall -y numpy
!pip install numpy --upgrade


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-2.2.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp312-cp312-macosx_11_0_arm64.whl (14.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.3 requires langchain-core>=0.3.52, but you have langchain-core 0.1.46 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.5 which is incompatible.
langchain-community 0.0.34 requires numpy<2,>=1, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.5 which is incomp

In [4]:
!pip install numpy==1.26.4


  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-chroma 0.2.3 requires langchain-core>=0.3.52, but you have langchain-core 0.1.46 which is incompatible.
gradio 5.20.0 requires pydantic>=2.0, but you have pydantic 1.10.22 which is incompatible.


In [5]:
!pip install openai
!pip install dotenv

In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the variables
KEY=os.getenv("OPENAI_KEY")

# Print (for debugging, avoid printing sensitive data in production)

In [7]:
!pip install openai

In [8]:
!pip install --upgrade openai

In [ ]:
from flask import Flask, request, jsonify, render_template
import pickle
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
# Initialize Flask App
app = Flask(__name__)

# Load the trained model and preprocessor
with open("../heart_model.pkl", "rb") as f:
     model = pickle.load(f)

with open("../preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

# to implement api
from dotenv import load_dotenv
import os

load_dotenv()
KEY = os.getenv('OPENAI_KEY')
if not KEY:
    raise ValueError('OpenAI API key not found in environment variables')

openai_client = openai
openai.api_key = KEY


@app.route("/")
def home():
    return render_template("index.html")

@app.route('/doctors')
def doctors():
    return render_template('doctors.html')

# Prediction Route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        raw_features = data["features"]

        # Convert numerical values from string to proper types
        processed_features = [
            int(raw_features[0]),  # Age
            raw_features[1],        # Gender (string, remains unchanged)
            raw_features[2],        # Chest Pain Type (string, remains unchanged)
            int(raw_features[3]),  # Resting BP
            int(raw_features[4]),  # Cholesterol
            int(raw_features[5]),  # Fasting BS (0 or 1)
            raw_features[6],        # Resting ECG (string, remains unchanged)
            int(raw_features[7]),  # Max HR
            raw_features[8],        # Exercise Angina (string, remains unchanged)
            float(raw_features[9]), # Oldpeak (decimal value)
            raw_features[10]        # ST Slope (string, remains unchanged)
        ]

        # Convert to DataFrame for transformation
        features_df = pd.DataFrame([processed_features], columns=preprocessor.feature_names_in_)

        # Transform using the preprocessor
        transformed_features = preprocessor.transform(features_df)

        # Make prediction
        prediction = model.predict(transformed_features)[0]
        probability = model.predict_proba(transformed_features)[0][1]

        return jsonify({
            "message": "Appointment is booked for you" if int(prediction) else "You don't need a doctor",
            "prediction": int(prediction),
            "probability": float(probability)
        })
    except Exception as e:
        return jsonify({"error": str(e)})

# Chatbot Route (Cardiac-focused Assistant)
@app.route("/chat", methods=["POST"])
def chat():
    try:
        user_input = request.json["message"]

        # Restrict chatbot to only discuss cardiac health
        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a health assistant specializing in cardiac conditions. You only provide advice on heart diseases, symptoms, risk factors, prevention, and treatment. If a user asks something unrelated, politely steer them back to heart health topics."},
                {"role": "user", "content": user_input}
            ]
        )

        return jsonify({"response": response.choices[0].message.content})
    except Exception as e:
        return jsonify({"response": "Error: " + str(e)})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8084)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8084
 * Running on http://192.168.133.119:8084
Press CTRL+C to quit
127.0.0.1 - - [06/May/2025 23:41:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2025 23:41:51] "GET /static/heart.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [06/May/2025 23:41:51] "GET /favicon.ico HTTP/1.1" 404 -
